In [30]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import timedelta
from datetime import date
import matplotlib.pyplot as plt



In [31]:
tickers = ["TATAMOTORS.NS","POLYCAB.NS","RELIANCE.NS"]
start_date= date.today()-timedelta(weeks=52)
end_date= date.today()-timedelta(weeks=4)

stock_data = []

for ticker in tickers:

    raw_data = yf.download(ticker,start_date,end_date,interval='1d')
    raw_data['Symbol']=ticker
    stock_data.append(raw_data)

stock_data = pd.concat(stock_data)
stock_data.reset_index(inplace=True)

stock_data


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,2023-08-21,617.000000,623.299988,614.799988,620.500000,618.591064,9580365,TATAMOTORS.NS
1,2023-08-22,621.799988,626.500000,618.599976,620.200012,618.291992,8534395,TATAMOTORS.NS
2,2023-08-23,622.750000,622.750000,613.150024,613.900024,612.011414,8248333,TATAMOTORS.NS
3,2023-08-24,618.000000,619.000000,609.650024,610.950012,609.070435,7096025,TATAMOTORS.NS
4,2023-08-25,610.849976,616.700012,593.299988,605.099976,603.238403,16075509,TATAMOTORS.NS
...,...,...,...,...,...,...,...,...
664,2024-07-12,3169.000000,3210.300049,3149.000000,3193.449951,3193.449951,6462392,RELIANCE.NS
665,2024-07-15,3208.899902,3211.699951,3171.000000,3194.449951,3194.449951,2664844,RELIANCE.NS
666,2024-07-16,3187.000000,3200.000000,3138.250000,3152.500000,3152.500000,4693419,RELIANCE.NS
667,2024-07-18,3152.000000,3187.800049,3115.649902,3173.350098,3173.350098,6149187,RELIANCE.NS


In [32]:
stock_data["WeekStartDate"] = stock_data["Date"] - pd.to_timedelta(stock_data["Date"].dt.dayofweek, unit='D')

stock_data['WeekDayNumber'] = stock_data["Date"].dt.weekday

grouped_df = stock_data.groupby(["Symbol","WeekStartDate"])["WeekDayNumber"].max().reset_index()

friday_close_data = pd.merge(stock_data,grouped_df,on=["Symbol","WeekStartDate","WeekDayNumber"],how="inner")

friday_close_data



,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,WeekStartDate,WeekDayNumber
0,2023-08-25,610.849976,616.700012,593.299988,605.099976,603.238403,16075509,TATAMOTORS.NS,2023-08-21,4
1,2023-09-01,604.200012,614.900024,601.900024,611.200012,609.319702,10297492,TATAMOTORS.NS,2023-08-28,4
2,2023-09-08,617.950012,628.900024,614.200012,627.250000,625.320312,16190749,TATAMOTORS.NS,2023-09-04,4
3,2023-09-15,628.700012,636.500000,627.150024,634.250000,632.298767,18917805,TATAMOTORS.NS,2023-09-11,4
4,2023-09-22,628.000000,630.799988,619.000000,620.450012,618.541260,6551917,TATAMOTORS.NS,2023-09-18,4
...,...,...,...,...,...,...,...,...,...,...
139,2024-06-21,2947.250000,2949.399902,2881.149902,2908.399902,2908.399902,15585180,RELIANCE.NS,2024-06-17,4
140,2024-06-28,3062.050049,3162.000000,3062.050049,3130.800049,3130.800049,14478668,RELIANCE.NS,2024-06-24,4
141,2024-07-05,3107.649902,3197.000000,3096.000000,3177.250000,3177.250000,6134855,RELIANCE.NS,2024-07-01,4
142,2024-07-12,3169.000000,3210.300049,3149.000000,3193.449951,3193.449951,6462392,RELIANCE.NS,2024-07-08,4


In [33]:
friday_close_data["FourWeeksAgo"] = friday_close_data["WeekStartDate"] - pd.DateOffset(weeks=4)

momentum_data = friday_close_data.merge(friday_close_data[["WeekStartDate","Symbol","Close"]],left_on=["Symbol","FourWeeksAgo"],right_on=["Symbol","WeekStartDate"],suffixes =("","_FourWeeksAgo"),how = "left")
momentum_data


,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,WeekStartDate,WeekDayNumber,FourWeeksAgo,WeekStartDate_FourWeeksAgo,Close_FourWeeksAgo
0,2023-08-25,610.849976,616.700012,593.299988,605.099976,603.238403,16075509,TATAMOTORS.NS,2023-08-21,4,2023-07-24,NaT,NaN
1,2023-09-01,604.200012,614.900024,601.900024,611.200012,609.319702,10297492,TATAMOTORS.NS,2023-08-28,4,2023-07-31,NaT,NaN
2,2023-09-08,617.950012,628.900024,614.200012,627.250000,625.320312,16190749,TATAMOTORS.NS,2023-09-04,4,2023-08-07,NaT,NaN
3,2023-09-15,628.700012,636.500000,627.150024,634.250000,632.298767,18917805,TATAMOTORS.NS,2023-09-11,4,2023-08-14,NaT,NaN
4,2023-09-22,628.000000,630.799988,619.000000,620.450012,618.541260,6551917,TATAMOTORS.NS,2023-09-18,4,2023-08-21,2023-08-21,605.099976
...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2024-06-21,2947.250000,2949.399902,2881.149902,2908.399902,2908.399902,15585180,RELIANCE.NS,2024-06-17,4,2024-05-20,2024-05-20,2960.500000
140,2024-06-28,3062.050049,3162.000000,3062.050049,3130.800049,3130.800049,14478668,RELIANCE.NS,2024-06-24,4,2024-05-27,2024-05-27,2860.800049
141,2024-07-05,3107.649902,3197.000000,3096.000000,3177.250000,3177.250000,6134855,RELIANCE.NS,2024-07-01,4,2024-06-03,2024-06-03,2939.899902
142,2024-07-12,3169.000000,3210.300049,3149.000000,3193.449951,3193.449951,6462392,RELIANCE.NS,2024-07-08,4,2024-06-10,2024-06-10,2955.100098


In [34]:
momentum_data["Momentum"] = 1+((momentum_data["Close"]-momentum_data["Close_FourWeeksAgo"])/momentum_data["Close_FourWeeksAgo"])
momentum_by_ticker = momentum_data.groupby(["Symbol"])["Momentum"].agg("prod")

momentum_by_ticker = momentum_by_ticker - 1
momentum_by_ticker = momentum_by_ticker.reset_index()
momentum_by_ticker

,Symbol,Momentum
0,POLYCAB.NS,1.669730
1,RELIANCE.NS,1.757184
2,TATAMOTORS.NS,5.728166


In [35]:
friday_close_data["OneWeekAgo"] = friday_close_data["WeekStartDate"] - pd.DateOffset(weeks=1)
momentum_data = friday_close_data.merge(friday_close_data[["WeekStartDate","Symbol","Close"]],left_on=["Symbol","OneWeekAgo"],right_on=["Symbol","WeekStartDate"],suffixes =("","_OneWeeksAgo"),how = "left")

momentum_data


,Date,Open,High,Low,Close,Adj Close,Volume,Symbol,WeekStartDate,WeekDayNumber,FourWeeksAgo,OneWeekAgo,WeekStartDate_OneWeeksAgo,Close_OneWeeksAgo
0,2023-08-25,610.849976,616.700012,593.299988,605.099976,603.238403,16075509,TATAMOTORS.NS,2023-08-21,4,2023-07-24,2023-08-14,NaT,NaN
1,2023-09-01,604.200012,614.900024,601.900024,611.200012,609.319702,10297492,TATAMOTORS.NS,2023-08-28,4,2023-07-31,2023-08-21,2023-08-21,605.099976
2,2023-09-08,617.950012,628.900024,614.200012,627.250000,625.320312,16190749,TATAMOTORS.NS,2023-09-04,4,2023-08-07,2023-08-28,2023-08-28,611.200012
3,2023-09-15,628.700012,636.500000,627.150024,634.250000,632.298767,18917805,TATAMOTORS.NS,2023-09-11,4,2023-08-14,2023-09-04,2023-09-04,627.250000
4,2023-09-22,628.000000,630.799988,619.000000,620.450012,618.541260,6551917,TATAMOTORS.NS,2023-09-18,4,2023-08-21,2023-09-11,2023-09-11,634.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,2024-06-21,2947.250000,2949.399902,2881.149902,2908.399902,2908.399902,15585180,RELIANCE.NS,2024-06-17,4,2024-05-20,2024-06-10,2024-06-10,2955.100098
140,2024-06-28,3062.050049,3162.000000,3062.050049,3130.800049,3130.800049,14478668,RELIANCE.NS,2024-06-24,4,2024-05-27,2024-06-17,2024-06-17,2908.399902
141,2024-07-05,3107.649902,3197.000000,3096.000000,3177.250000,3177.250000,6134855,RELIANCE.NS,2024-07-01,4,2024-06-03,2024-06-24,2024-06-24,3130.800049
142,2024-07-12,3169.000000,3210.300049,3149.000000,3193.449951,3193.449951,6462392,RELIANCE.NS,2024-07-08,4,2024-06-10,2024-07-01,2024-07-01,3177.250000


In [36]:
momentum_data["Smoothness"] = momentum_data["Close"] - momentum_data["Close_OneWeeksAgo"]
smo_by_ticker = momentum_data.groupby("Symbol").agg(Smoothness=("Smoothness",lambda x: (x>0).sum() / (len(x)-1)))

smo_by_ticker

,Smoothness
Symbol,
POLYCAB.NS,0.595745
RELIANCE.NS,0.574468
TATAMOTORS.NS,0.702128


In [37]:
momentum_smooth_by_ticker = momentum_by_ticker.merge(smo_by_ticker, on="Symbol", how="inner")

momentum_smooth_by_ticker


,Symbol,Momentum,Smoothness
0,POLYCAB.NS,1.669730,0.595745
1,RELIANCE.NS,1.757184,0.574468
2,TATAMOTORS.NS,5.728166,0.702128
